In [50]:
import pandas as pd

df = pd.read_csv('n_movies.csv', encoding='latin-1', skiprows=1001, nrows=1000)


In [51]:
df

,Spy Kids 3: Game Over,(2003),PG,84 min,"Action, Adventure, Comedy",4.3,"Carmen's caught in a virtual reality game designed by the Kids' new nemesis, the Toymaker. It's up to Juni to save his sister, and ultimately the world.","['Robert Rodriguez', '| ', ' Stars:', 'Daryl Sabara, ', 'Alexa PenaVega, ', 'Antonio Banderas, ', 'Carla Gugino']","60,653"
0,All the Bright Places,(2020),TV-MA,107 min,"Drama, Romance",6.5,"The story of Violet and Theodore, who meet and...","['Brett Haley', '| ', ' Stars:', 'Elle Fann...","31,536"
1,A Classic Horror Story,(2021),TV-MA,95 min,"Drama, Horror, Mystery",5.7,"In this gruesome suspense film, strangers trav...","['Roberto De Feo, ', 'Paolo Strippoli', '| ', ...","18,171"
2,Resident Evil: The Final Chapter,(2016),R,107 min,"Action, Horror, Sci-Fi",5.5,Alice returns to where the nightmare began: Th...,"['Paul W.S. Anderson', '| ', ' Stars:', 'Mi...","94,352"
3,Once Upon a Time in Mexico,(2003),R,102 min,"Action, Crime, Thriller",6.3,"Hitman ""El Mariachi"" becomes involved in inter...","['Robert Rodriguez', '| ', ' Stars:', 'Anto...","162,527"
4,El Chapo,(2017â2018),TV-MA,43 min,"Biography, Crime, Drama",7.8,"A look at the life of notorious drug kingpin, ...","['Marco de la O, ', 'Humberto Busto, ', 'Diego...","18,453"
...,...,...,...,...,...,...,...,...,...
995,Bulgasal,(2021â ),NaN,60 min,"Drama, Fantasy, Thriller",7.3,"The story of a man-turned-immortal, Bulgasal, ...","['Lee Jin-Wook, ', 'Nara, ', 'Joon Lee, ', 'Go...","2,280"
996,Dragon's Lair: The Movie,NaN,NaN,NaN,"Animation, Action, Adventure",NaN,"Dirk the Daring, a knight who sets out to resc...","['Don Bluth, ', 'Gary Goldman', '| ', ' Sta...",NaN
997,Tramps,(2016),NaN,82 min,"Adventure, Romance",6.5,A young man and woman find love in an unlikely...,"['Adam Leon', '| ', ' Stars:', 'Callum Turn...","5,721"
998,I Lost My Body,(2019),TV-MA,81 min,"Animation, Drama, Fantasy",7.5,"A story of Naoufel, a young man who is in love...","['JÃ©rÃ©my Clapin', '| ', ' Stars:', 'Hakim...","32,912"


In [48]:
def text_representation(row):
    text_rep = f"""
    Title: {row['title']},
    Genre: {row['genre']},
    Description: {row['description']}"""
    
    return text_rep
    

In [45]:
df['text_rep']=df.apply(text_representation,axis =1)

KeyError: 'title'

In [4]:
import faiss
import requests
import numpy as np

In [7]:
dim = 4096
index = faiss.IndexFlatL2(dim)
x = np.zeros((len(df['text_rep']),dim),dtype ='float32')


In [8]:
for i, rep in enumerate(df['text_rep']):
    if i%100 == 0:
        print('Processed',str(i),'instances')
    res = requests.post('http://localhost:11434/api/embeddings',
        json={
            'model':'llama2',
            'prompt': rep
        
        }
    )
    embedding = res.json()['embedding']
    x[i] = np.array(embedding)
index.add(x)


Processed 0 instances
Processed 100 instances
Processed 200 instances
Processed 300 instances
Processed 400 instances
Processed 500 instances
Processed 600 instances
Processed 700 instances
Processed 800 instances
Processed 900 instances


In [9]:
faiss.write_index(index,'index')

In [10]:
index = faiss.read_index('index')

In [11]:
df[df.title.str.contains('Horror')]

,title,year,certificate,duration,genre,rating,description,stars,votes,text_rep
23,American Horror Story,(2011â ),TV-MA,60 min,"Drama, Horror, Sci-Fi",8.0,An anthology series centering on different cha...,"['Lady Gaga, ', 'Kathy Bates, ', 'Angela Basse...","312,970","Certificate: TV-MA,\n Title: American Horro..."


In [36]:
favorite_movie = df.iloc[23]

representation ="""Certificate: TV-MA,
    Title: Titans,
    Year: (2011â ),
    Stars: ['Younghoe Koo'],
    Rating: 9.0,
    Genre: Action,
    Description: A small village is attacked by monsters"""

In [37]:
res = requests.post('http://localhost:11434/api/embeddings',
        json={
            'model':'llama2',
            'prompt': representation
        
        })
    

In [39]:
embedding = np.array([res.json()['embedding']],dtype = 'float32')

D, I = index.search(embedding,5)


In [40]:
# Retrieve embeddings for top matches
best_match_embeddings = np.array([x[i] for i in I.flatten()], dtype='float32')

# Retrieve the movie details from the DataFrame
best_match_movies = df.iloc[I.flatten()]

print(best_match_movies)

                           title           year certificate duration  \
555                      Between  (2015â2016)       TV-14   60 min   
688  The Festival of Troubadours         (2022)       TV-14  102 min   
501   Avatar: The Last Airbender            NaN         NaN      NaN   
516               Enola Holmes 2         (2022)       PG-13      NaN   
150                The Good Wife  (2009â2016)       TV-14   43 min   

                         genre  rating  \
555     Drama, Mystery, Sci-Fi     5.9   
688                      Drama     5.9   
501  Action, Adventure, Comedy     NaN   
516   Action, Adventure, Crime     NaN   
150      Crime, Drama, Mystery     8.4   

                                           description  \
555  After a mysterious disease kills every residen...   
688  An unexpected reunion between a traveling musi...   
501  Live-action adaptation of the animated series ...   
516  Now a detective-for-hire, Enola Holmes takes o...   
150  Alicia Florrick has